# Resarch Paper Summarizer by Name

### Imports

In [0]:
## If dependencies do not exist please install them
# !pip install python-dotenv openai arxiv

In [0]:
import arxiv
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

### Load Open AI Key

In [0]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("❌ No OpenAI API key found in .env file.")
else:
    print("✅ API key loaded successfully.")

# ✅ Initialize OpenAI
openai = OpenAI(api_key=api_key)

### Main Class MLResearchFetcher

In [0]:
class MLResearchFetcher:
    def __init__(self, system_prompt, query="machine learning", max_results=5):
        self.query = query
        self.max_results = max_results
        self.system_prompt = system_prompt

    def fetch_papers(self):
        search = arxiv.Search(
            query=f'ti:"{self.query}"',
            max_results=self.max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate,
            sort_order=arxiv.SortOrder.Descending,
        )
        return list(search.results())

    def summarize_abstract(self, abstract, system_prompt):
        try:
            completion = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": abstract}
                ]
            )
            return completion.choices[0].message.content.strip()
        except Exception as e:
            return f"❌ Error during summarization: {e}"

    def display_results(self):
        papers = self.fetch_papers()
        for paper in papers:
            display(Markdown(f"### 📄 [{paper.title}]({paper.entry_id})"))
            display(Markdown(f"**Authors:** {', '.join(author.name for author in paper.authors)}"))
            display(Markdown(f"**Published:** {paper.published.date()}"))
            display(Markdown(f"**Abstract:** {paper.summary.strip()}"))
            summary = self.summarize_abstract(paper.summary, self.system_prompt)
            display(Markdown(f"**🔍 Summary:** {summary}"))
            display(Markdown("---"))

### Helper Functions

In [0]:
system_prompt = "You are an expert research paper summarizer and AI research assistant. \
When provided with the URL or content of a research paper in the field of machine learning, artificial intelligence, or data science, perform the following: \
1. **Extract and present** the following details in a clear, structured Markdown format: \
   - Title and Author(s) \
   - Year of Publication \
   - Objective or Aim of the Research (Why the study was conducted) \
   - Background or Introduction (What foundational knowledge or motivation led to this work) \
   - Type of Research (e.g., empirical study, theoretical analysis, experimental benchmark) \
   - Methods or Methodology (How the research was conducted: dataset, models, techniques used) \
   - Results and Key Findings (What was discovered or proven) \
   - Conclusion (Summary of insights, limitations, and proposed future work) \
\
2. **Evaluate** the impact and relevance of the paper: \
   - Assess the significance of the research to the broader ML/AI community \
   - Note any novelty, performance improvements, or theoretical breakthroughs \
   - Comment on the potential applications or industry relevance \
\
3. **Suggest new research directions**: \
   - Identify gaps, limitations, or unexplored ideas in the paper \
   - Propose at least one new research idea or follow-up paper that builds upon this work \
\
Respond in a clean, professional Markdown format suitable for researchers or students reviewing the literature."


In [0]:
def print_papers(papers):
    for paper in papers:
        title = paper.title
        authors = ", ".join(author.name for author in paper.authors)
        published = paper.published.strftime('%Y-%m-%d')
        abstract = paper.summary.strip()
        link = paper.entry_id
        pdf_link = [l.href for l in paper.links if l.title == 'pdf']
        categories = ", ".join(paper.categories)

        print(f"\n📄 Title: {title}")
        print(f"👥 Authors: {authors}")
        print(f"📅 Published: {published}")
        print(f"🏷️ Categories: {categories}")
        print(f"🔗 Link: {link}")
        if pdf_link:
            print(f"📄 PDF: {pdf_link[0]}")
        print(f"\n📝 Abstract:\n{abstract}")
        print("-" * 80)


#### Get the papers given the name of the paper

In [0]:
fetcher = MLResearchFetcher(system_prompt, query="QWEN2 TECHNICAL REPORT", max_results=3)
papers = fetcher.fetch_papers()
print_papers(papers)

### Call the model and get the results

In [0]:
fetcher.display_results()